In [1]:
import mido
from mido import Message, MidiFile, MidiTrack, bpm2tempo

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------
output_path = r"C:\Users\magst\OneDrive\Pictures\Desktop\NICO\NICOTEST.mid"

bpm = 110                # tempo, adjust as you like
bars = 4                # total bars
beats_per_bar = 4        # 4/4 time
ticks_per_beat = 480     # MIDI resolution

midi_key = 43            # base MIDI note (43 = G2, a nice bass register)
velocity = 90            # note intensity
channel = 0              # MIDI channel for bass

# Simple bass pattern (in beats) relative to midi_key
# You can change these intervals to match any groove you like
# Format: (semitone_offset_from_key, duration_in_beats)
bass_pattern = [
    (0,   1.0),   # root
    (3,   1.0),   # minor third
    (5,   1.0),   # fourth
    (7,   1.0),   # fifth
]

# -------------------------------------------------------------------
# BUILD MIDI FILE
# -------------------------------------------------------------------
mid = MidiFile(type=1)
mid.ticks_per_beat = ticks_per_beat

track = MidiTrack()
mid.tracks.append(track)

# Set tempo
tempo = bpm2tempo(bpm)  # microseconds per beat
track.append(mido.MetaMessage('set_tempo', tempo=tempo, time=0))

# Optional: set instrument to a bass patch (program 33 = fingered bass on GM)
track.append(Message('program_change', program=32, channel=channel, time=0))

current_time_ticks = 0

# Total beats in the song
total_beats = bars * beats_per_bar

# We’ll fill the entire duration with repetitions of bass_pattern
pattern_beats = sum(d for _, d in bass_pattern)
num_repeats = int(total_beats // pattern_beats)

for _ in range(num_repeats):
    for interval, dur_beats in bass_pattern:
        note = midi_key + interval
        dur_ticks = int(dur_beats * ticks_per_beat)

        # Note on
        track.append(
            Message(
                'note_on',
                note=note,
                velocity=velocity,
                channel=channel,
                time=0  # delta from previous event on same track
            )
        )

        # Note off after dur_ticks
        track.append(
            Message(
                'note_off',
                note=note,
                velocity=0,
                channel=channel,
                time=dur_ticks
            )
        )

        current_time_ticks += dur_ticks

# If total_beats wasn't an exact multiple of pattern length, you could
# optionally pad with rests here, but for most tempos/patterns this is fine.

mid.save(output_path)
print(f"Saved MIDI file to: {output_path}")


Saved MIDI file to: C:\Users\magst\OneDrive\Pictures\Desktop\NICO\NICOTEST.mid


In [2]:
import pygame

midi_path = r"C:\Users\magst\OneDrive\Pictures\Desktop\NICO\NICOTEST.mid"

pygame.init()
pygame.mixer.init()

# Load & play
pygame.mixer.music.load(midi_path)
pygame.mixer.music.play()

print("Playing MIDI...")

# Keep Python alive until it finishes
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)


pygame 2.6.1 (SDL 2.28.4, Python 3.10.15)
Hello from the pygame community. https://www.pygame.org/contribute.html
Playing MIDI...


In [ ]:
pip install pygame